In [3]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('delivery.csv', 'delivery'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items')
]

# MySQL connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Bhanu0843@',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder path
folder_path = 'E:/Data Analysis/E-Commerce/Dataset'

# Function to map pandas dtypes to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

# Loop through each file and process
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    if not os.path.exists(file_path):
        print(f"⚠️ File not found: {file_path}")
        continue

    print(f"📥 Processing: {csv_file}")

    # Load CSV with header handling and clean data
    df = pd.read_csv(file_path, header=0, skip_blank_lines=True)

    # Drop columns with all NaN values
    df.dropna(axis=1, how='all', inplace=True)

    # Clean and sanitize column names
    df.columns = [str(col).strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]
    df.columns = ['col_' + str(i) if str(col).lower() == 'nan' else col for i, col in enumerate(df.columns)]

    # Debug: Show column names
    print(f"📊 Columns in `{table_name}`: {df.columns.tolist()}")

    # Replace NaN with None for SQL NULL
    df = df.where(pd.notnull(df), None)

    # Drop table if it exists (to avoid structure mismatch)
    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")

    # Create table
    column_definitions = ', '.join([f'`{col}` {get_sql_type(dtype)}' for col, dtype in df.dtypes.items()])
    create_table_query = f"CREATE TABLE `{table_name}` ({column_definitions})"
    cursor.execute(create_table_query)

    # Prepare INSERT query
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({placeholders})"

    # Insert rows
    inserted_rows = 0
    for row in df.itertuples(index=False, name=None):
        try:
            cursor.execute(insert_query, row)
            inserted_rows += 1
        except Exception as e:
            print(f"❌ Error inserting into `{table_name}`: {e}")
            print(f"⛔ Problematic row: {row}")

    # Commit changes for this table
    conn.commit()
    print(f"✅ Inserted {inserted_rows} records into `{table_name}`.\n")

# Close connections
cursor.close()
conn.close()
print("🎉 All done! MySQL connection closed.")


📥 Processing: customers.csv
📊 Columns in `customers`: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
✅ Inserted 99441 records into `customers`.

📥 Processing: orders.csv
📊 Columns in `orders`: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
✅ Inserted 99441 records into `orders`.

📥 Processing: sellers.csv
📊 Columns in `sellers`: ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']
✅ Inserted 3095 records into `sellers`.

📥 Processing: products.csv
📊 Columns in `products`: ['product_id', 'product_category', 'product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
❌ Error inserting into `products`: 1054 (42S22): Unknown column 'nan' in 'field list'
⛔ Problematic row: ('a41e356c

In [ ]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
   ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items')  # Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Bhanu0843@',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'E:/Data Analysis/E-Commerce/Dataset'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Data Analysis/E-Commerce/Dataset\\delivery.csv'